In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [3]:
file_name="TrainingSet.pkl"
raw_data = pd.read_pickle(file_name)
#print(raw_data)

data = raw_data
print("In this dataset we have columns of:")
print('Co = Coordinates array 10x3')
print('Ce = Cell paramaters array 3x3')
print('X = Representation array 4x2')
print('Z = Atoms (number in periodic table in format down,along from 1 without transition Metals) int')
print('T = Total Formation Energy int')
print('N = Number of Atoms int')

#print(dataset['Z'])
#print((pd.Series(data['Ce'][:,0, 2])))
#data['Co'][0]
data['X'][2], data['Z'][0]

In this dataset we have columns of:
Co = Coordinates array 10x3
Ce = Cell paramaters array 3x3
X = Representation array 4x2
Z = Atoms (number in periodic table in format down,along from 1 without transition Metals) int
T = Total Formation Energy int
N = Number of Atoms int


(array([[3, 2],
        [3, 6],
        [3, 3],
        [5, 3]], dtype=int64),
 array([51, 51, 51, 51, 51, 51,  6,  8, 54, 54], dtype=int64))

In [4]:
co_index = pd.MultiIndex.from_product(
    (("X1", "X2", "X3", "X4", "X5", "X6", "A1", "A2", "B1", "B2"), ("x", "y", "z")))
#co_index

dataframe = pd.DataFrame([np.ravel(d) for d in data['Co']], columns=co_index)
dataframe['Cell Parameter'] = pd.Series([d[0][1] for d in data['Ce']], name="cell_parameter")


rep_index = pd.MultiIndex.from_product((('Atom 1', 'Atom 2', 'Atom 3', 'Atom 4'), ('XPT', 'YPT')))
X_frame = pd.DataFrame([np.ravel(d) for d in data['X']], columns=rep_index)
dataframe = pd.concat([dataframe, X_frame], axis=1)

dataframe['Formation Energy'] = data['T']
#print(dataframe)

In [5]:
def multi_lin_reg(data, test_size):
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    np.set_printoptions(precision=2)
    return(round(np.sqrt(sk.metrics.mean_squared_error(y_test, y_pred)), 3)) #rms error

def poly_reg(data, test_size, poly_degree):
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)
    poly_reg = PolynomialFeatures(degree = poly_degree)
    X_poly = poly_reg.fit_transform(X_train)
    lin_reg = LinearRegression()
    lin_reg.fit(X_poly, y_train)
    y_pred = lin_reg.predict(poly_reg.fit_transform(X_test))
    return(round(np.sqrt(sk.metrics.mean_squared_error(y_test, y_pred)), 3))

def random_forest_reg(data, test_size, num_estimators):
    X = dataframe.iloc[:, 1:-1].values
    y = dataframe.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)
    regressor = RandomForestRegressor(n_estimators = num_estimators, random_state = 0)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict((X_test))
    return(round(np.sqrt(sk.metrics.mean_squared_error(y_test, y_pred)), 3))

def ann_reg(data, train_split, units_1, units_2, batch_size, epochs):
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = train_split, random_state = 0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # https://stackoverflow.com/questions/49008074/how-to-create-a-neural-network-for-regression - was useful here
    ann = tf.keras.models.Sequential()
    
    ann.add(tf.keras.layers.Dense(units=units_1, activation='relu'))
    
    ann.add(tf.keras.layers.Dense(units=units_2, activation='sigmoid'))
    
    ann.compile(optimizer = 'adam', loss = 'mean_squared_error')
    
    ann.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, verbose = 0)
    
    y_pred = ann.predict(X_test)
    
    return(round(np.sqrt(sk.metrics.mean_squared_error(y_test, y_pred)), 3))

In [6]:
print(multi_lin_reg(dataframe, 0.2))
print(poly_reg(dataframe, 0.2, 1))
print(random_forest_reg(dataframe, 0.2, 10))
print(ann_reg(dataframe, 0.2, 6, 1, 32, 20))

8.584
8.645
3.081
10.477


In [5]:
#PREVIOUS WORKINGS FOR REFERENCE

X_x = pd.Series(data['Ce'][:,0, 0])   #Please note I have assigned these X,B,A and x,y,z randomly as I am not certain which are
X_y = pd.Series(data['Ce'][:,0, 1])   #which. These are just labels at the moment. 
X_z = pd.Series(data['Ce'][:,0, 2])

B_x = pd.Series(data['Ce'][:,1, 0])
B_y = pd.Series(data['Ce'][:,1, 1])
B_z = pd.Series(data['Ce'][:,1, 2])

A_x = pd.Series(data['Ce'][:,2, 0])
A_y = pd.Series(data['Ce'][:,2, 1])
A_z = pd.Series(data['Ce'][:,2, 2])

E = pd.Series(data['T'])

dic = {'X_length x': X_x, 'X length y': X_y, 'X length z': X_z, 
       'B length x': B_x, 'B length y': B_y, 'B length z': B_z, 
       'A length x': A_x, 'A length y': A_y, 'A length z': A_z, ' Formation Energy': E}

frame = pd.DataFrame(dic)
frame.reindex
frame                               #I am also not sure why three of these are assigned to 0, and why they're the same number.

,X_length x,X length y,X length z,B length x,B length y,B length z,A length x,A length y,A length z,Formation Energy
0,0.0,5.130330,5.130330,5.130330,0.0,5.130330,5.130330,5.130330,0.0,9.603148
1,0.0,6.897225,6.897225,6.897225,0.0,6.897225,6.897225,6.897225,0.0,-7.110018
2,0.0,5.359070,5.359070,5.359070,0.0,5.359070,5.359070,5.359070,0.0,4.713586
3,0.0,5.642745,5.642745,5.642745,0.0,5.642745,5.642745,5.642745,0.0,18.374580
4,0.0,4.729090,4.729090,4.729090,0.0,4.729090,4.729090,4.729090,0.0,13.204600
...,...,...,...,...,...,...,...,...,...,...
10585,0.0,5.518195,5.518195,5.518195,0.0,5.518195,5.518195,5.518195,0.0,13.389259
10586,0.0,5.291100,5.291100,5.291100,0.0,5.291100,5.291100,5.291100,0.0,-3.022594
10587,0.0,5.570835,5.570835,5.570835,0.0,5.570835,5.570835,5.570835,0.0,0.171113
10588,0.0,5.277365,5.277365,5.277365,0.0,5.277365,5.277365,5.277365,0.0,11.400947
